In [ ]:
from langchain import HuggingFacePipeline, PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [ ]:
# GPUの確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\n!!! current device is {device} !!!\n")

In [ ]:
# モデルのダウンロード
model_id = "inu-ai/dolly-japanese-gpt-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)
# model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
# LLMs: langchainで上記モデルを利用する
task = "text-generation"
pipe = pipeline(
    task, 
    model=model,
    tokenizer=tokenizer,
    device=0,
    framework='pt',
    max_new_tokens=32,
    temperature=0.1,
)

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# Prompts: プロンプトを作成
template = r"<s>\n以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n[SEP]\n指示:\n{instruction}\n[SEP]\n入力:\n{question}\n[SEP]\n応答:\n"
# template = """\n以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n指示:\n{instruction}\n\n入力:\n{question}\n\n応答:\n"""
prompt = PromptTemplate(template=template, input_variables=["instruction", "question"])

# Chains: llmを利用可能な状態にする
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)

In [ ]:
# 指示を与え、質問を投げる
import time
instruction = "あなたはユーザの入力に応答するアシスタントです。文章の繰り返しを避け、簡潔に要約して回答してください。"
question = "東京ってなに？"

start = time.time()

generated_text = llm_chain.run({"instruction":instruction, "question":question})

print(time.time()-start)
print(generated_text)

